<!--Información del curso-->
<img align="left" style="padding-right:10px;" src="figuras/logo_ciencia_datos.png">


<center><h2 style="font-size:2em;color:#840700">  Manejo de datos faltantes  </h4></center>

<br>
<table>
<col width="550">
<col width="450">
<tr>
<td><img src="figuras/nan_presentacion.png" align="left" style="width:500px"/></td>
<td>

* **Wes McKinney**, empezó a desarrollar Pandas en el año 2008 mientras trabajaba en *AQR Capital* [https://www.aqr.com/] por la necesidad que tenía de una herramienta flexible de alto rendimiento para realizar análisis cuantitativos en datos financieros. 
* Antes de dejar AQR convenció a la administración de la empresa de distribuir esta biblioteca bajo licencia de código abierto.
* **Pandas** es un acrónimo de **PANel DAta analysiS**
   
    
<br>
</td>
</tr>
</table>

# Librerias

In [253]:
import numpy as np
import matplotlib.pyplot as plt

#  Introducción

La diferencia entre los datos encontrados en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez estan limpios y homogéneos.
En particular, muchos conjuntos de datos interesantes tendrán cierta cantidad de datos faltantes. Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar datos faltantes de diferentes maneras.

En esta lección se discutirán algunas consideraciones generales para los datos faltantes, discutiremos cómo Pandas elige representarlos y se mostrarán algunas herramientas integradas de Pandas para manejar datos faltantes en Python.
Se hará referencia a los datos faltantes de manera general como valores * nulo *, * NaN * o * NA *.

# Datos faltantes en pandas

En el desarrollo de Pandas se eligió usar centinelas para los datos faltantes, y se eligió usar dos valores nulos de Python ya existentes: el valor especial de punto flotante **NaN** y el objeto Python **None**. Esta elección tiene algunos efectos secundarios, pero en la práctica termina siendo una buena elección en la mayoría de los casos de interés.

###  ``None``: Para datos faltantes de Python

El primer valor centinela utilizado por Pandas es ``None``, un objeto simple de Python que a menudo se usa para datos faltantes en el código de Python. Debido a que es un objeto Python, ``None`` no se puede usar en un arreglo número de de NumPy/Pandas.

### ``NaN``: Dato número faltante
 ``NaN`` es el acrónimo de ***Not a Number***. Es un valor especial de punto flotante reconocido por todos los sistemas que utilizan la representación estándar de punto flotante IEEE:

In [254]:
import numpy as np
import pandas as pd

valores = np.array([1, np.nan, 3, 4]) 
valores.dtype

dtype('float64')

NumPy eligió un tipo de punto flotante nativo para esta matriz: esto significa que, este arreglo admite operaciones como cualquier otro arreglo. **Debe tener en cuenta que `` NaN`` es  como un virus de datos: infecta cualquier otro objeto que toque**. Independientemente de la operación, el resultado de la aritmética con ``NaN`` será otro ``NaN``:

In [255]:
1 + np.nan

nan

In [256]:
0 *  np.nan

nan

Tenga en cuenta que esto significa que los **agregados** sobre los arreglos no generarán un error  pero no se tendrán resultados útiles:

In [257]:
valores.sum(), valores.min(), valores.max()

(np.float64(nan), np.float64(nan), np.float64(nan))

NumPy proporciona algunos **agregados** especiales que ignorarán estos valores faltantes:

In [258]:
np.nansum(valores), np.nanmin(valores), np.nanmax(valores)

(np.float64(8.0), np.float64(1.0), np.float64(4.0))

Importante: tenga en cuenta que ``NaN`` es específicamente un valor de punto flotante; no hay un valor equivalente de NaN para enteros, strings u otros tipos.

### NaN y None en Pandas

``NaN`` y ``None`` tienen su lugar, y Pandas está diseñado para manejarlos de una sola manera:

In [259]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

El enfoque de centinela  de Pandas funciona bastante bien en la práctica y rara vez causa problemas. 

## Operando con valores nulos

Como se ha visto, Pandas trata ``None`` y ``NaN`` de igual manera para indicar valores faltantes o nulos.
Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Son:

- ``isnull ()``: genera una máscara booleana que indica valores faltantes
- ``notnull ()``: opuesto a ``isnull ()``
- ``dropna ()``: Devuelve una versión filtrada de los datos
- ``fillna ()``: Devuelve una copia de los datos con valores faltantes rellenados 

### Detectando valores nulos
Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull ()`` y ``notnull ()``.
Cualquiera de los dos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [260]:
data = pd.Series([1, np.nan, 'hello', None])

In [261]:
# Aplicar la función isnull a la serie data
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [262]:
# Aplicar la función notnull a la serie data
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

Se puede por tanto generarar máscaras booleanas para un ``Serie`` o un ``DataFrame``

In [263]:
#Generar una serie de elementos no nulos de la serie data
data[data.notnull()]

0        1
2    hello
dtype: object

In [264]:
#Generar una serie de elementos nulos  de la serie data
data[data.isnull()]

1     NaN
3    None
dtype: object

### Descartando valores nulos

Además de las máscaras utilizadas anteriormente, existen los siguientes métodos: ``dropna ()``
(que elimina los valores de NA) y ``fillna ()`` (que completa los valores de NaN). Para una ``Serie``,
El resultado es sencillo:

In [265]:
#Aplicar la funcion dropna a la serie data
data.dropna()

0        1
2    hello
dtype: object

Para un ``DataFrame``, hay más opciones. Considere el siguiente ``DataFrame``:

In [266]:
df = pd.DataFrame([[1,      np.nan, 2.],
                   [2,      3,      5.],
                   [np.nan, 4,      6.]])
df

,0,1,2
0,1.0,NaN,2.0
1,2.0,3.0,5.0
2,NaN,4.0,6.0


No se puede eliminar valores individuales de un ``DataFrame``; solo se puede eliminar filas completas o columnas completas.
Dependiendo del problema que se tenga, es posible que desee uno u otro, por lo que ``dropna ()`` ofrece una serie de opciones para un ``DataFrame``. 

Por defecto, ``dropna ()`` eliminará todas las filas en las que *cualquier* valor nulo (NaN) esté presente:

In [267]:
#dropna()
df.dropna()

,0,1,2
1,2.0,3.0,5.0


Alternativamente, puede eliminar los valores de NaN a lo largo de un eje; ``axis = 1`` elimina todas las columnas que contienen un valor nulo:

In [268]:
#dropna(axis=1)
df.dropna(axis=1)

,2
0,2.0
1,5.0
2,6.0


In [269]:
# Puede usar axis='columns' para decir explicitamente que solo se afetaràn las columnas
# dropna(axis='columns')
df.dropna(axis='columns')

,2
0,2.0
1,5.0
2,6.0


In [270]:
# Puede usar axis='rows' para decir explicitamente que solo se afetaràn las fila
# dropna(axis='rows')
df.dropna(axis='rows')

,0,1,2
1,2.0,3.0,5.0


Pero esto también puede eliminar datos importantes; es posible eliminar solo filas o columnas que cumplan con alguna condición. Esto se puede especificar a través de los parámetros ``how`` o ``thresh``, que permiten un control preciso del número de elementos nulos para eliminar.

El valor predeterminado es ``how='any'``, de modo que cualquier fila o columna (dependiendo de la palabra clave ``axis``) que contenga un valor nulo se eliminará. También se puede especificar ``how='all'``, que solo eliminará filas/columnas que tengan  son *todos* sus valores nulos:


In [271]:
# Agrega al DataFrame,df, una nueva columna llamada 3 y que todos sus los elementos sean nulos (np.nan)
df['3'] = np.nan
df

,0,1,2,3
0,1.0,NaN,2.0,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN


In [272]:
#Eliminar las columnas que tengan todos sus elementos nulos



In [273]:
#Eliminar las columnas que contengan al menos un elemento nulo

df.dropna(axis='columns')

,2
0,2.0
1,5.0
2,6.0


Para un control más cuidadoso, el parámetro ``thresh``  permite especificar un número mínimo de valores **no nulos** para las filas/columnas que se mantendrán:

In [274]:
df[4]= pd.DataFrame([ 1.0,2.0, 3.0])
df[5]= pd.DataFrame([ 1.0,2.0, np.nan])
df[6]= pd.DataFrame([ 1.0,np.nan, np.nan])
df[7]= pd.DataFrame([ np.nan,np.nan, np.nan])

df

,0,1,2,3,4,5,6,7
0,1.0,NaN,2.0,NaN,1.0,1.0,1.0,NaN
1,2.0,3.0,5.0,NaN,2.0,2.0,NaN,NaN
2,NaN,4.0,6.0,NaN,3.0,NaN,NaN,NaN


1. Para las columnas

In [275]:
#thresh = 1
df.dropna(axis=1, thresh=1)

,0,1,2,4,5,6
0,1.0,NaN,2.0,1.0,1.0,1.0
1,2.0,3.0,5.0,2.0,2.0,NaN
2,NaN,4.0,6.0,3.0,NaN,NaN


In [276]:
#thresh = 2
df.dropna(axis=1, thresh=2)

,0,1,2,4,5
0,1.0,NaN,2.0,1.0,1.0
1,2.0,3.0,5.0,2.0,2.0
2,NaN,4.0,6.0,3.0,NaN


In [277]:
#thresh = 3
df.dropna(axis=1, thresh=3)

,2,4
0,2.0,1.0
1,5.0,2.0
2,6.0,3.0


2. Para las filas

In [278]:
#thresh = 3
 

In [279]:
#thresh = 4
 

Importante: Recuerde que ``thresh``  se refiere a valores **no nulos**

### Reemplazando  valores nulos

A veces, en lugar de descartar los valores NaN, convendría mejor reemplazarlos con un valor válido. *Este valor podría ser un número único como cero, o podría ser algún tipo de asignación o interpolación de los buenos valores.* Pandas proporciona el método ``fillna()``, que devuelve una copia de la matriz con valores nulos reemplazados.

Considere la siguiente ``Serie``:


In [288]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Podemos llenar entradas de NA con un solo valor, como cero:

In [289]:
#Rellenar con 0
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [291]:
#Rellenar con -10
data.fillna(-10)

a     1.0
b   -10.0
c     2.0
d   -10.0
e     3.0
dtype: float64

Se puede especificar un relleno hacia adelante (forward-fill) , para propagar el valor anterior hacia adelante:

In [ ]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

O se puede especificar un relleno para propagar los siguientes valores hacia atrás (back-fill ):

In [299]:
# back-fill
data.bfill()

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Para los *DataFrames*, las opciones son similares, pero también podemos especificar un eje,``axis``, a lo largo del cual tienen lugar el relleno:


In [285]:
df.columns=list('abcdefgh')
df

,a,b,c,d,e,f,g,h
0,1.0,NaN,2.0,NaN,1.0,1.0,1.0,NaN
1,2.0,3.0,5.0,NaN,2.0,2.0,NaN,NaN
2,NaN,4.0,6.0,NaN,3.0,NaN,NaN,NaN


In [286]:
#fillna(method='ffill', axis=1)


Observe que si un valor anterior no está disponible durante un llenado hacia adelante, el valor NA permanece.

In [287]:
# fillna(method='ffill', axis=0)


# Ejercicios

<div class="alert alert-info">
    
1. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, 90, np.nan, 95], 
        'Second Score': [30, 45, 56, np.nan], 
        'Third Score':[np.nan, 40, 80, 98]} 

    
<img align="left" width="350"  float= "none" align="middle" src="figuras/nan1.png">

    
</div> 

<div class="alert alert-info">
    
2. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, 90, np.nan, 95], 
        'Second Score': [30, 45, 56, np.nan], 
        'Third Score':[np.nan, 40, 80, 98]} 

    
<img align="left" width="350"  float= "none" align="middle" src="figuras/nan2.png">
    
</div> 

<div class="alert alert-info">
    
3. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, 90, np.nan, 95], 
        'Second Score': [30, np.nan, 45, 56], 
        'Third Score':[52, 40, 80, 98], 
        'Fourth Score':[np.nan, np.nan, np.nan, 65]} 
    
    
    
<img align="left" width="450"  float= "none" align="middle" src="figuras/nan3.png">

</div> 

<div class="alert alert-info">
    
4. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, np.nan, np.nan, 95], 
        'Second Score': [30, np.nan, 45, 56], 
        'Third Score':[52, np.nan, 80, 98], 
        'Fourth Score':[np.nan, np.nan, np.nan, 65]} 

    
    
<img align="left" width="350"  float= "none" align="middle" src="figuras/nan4.png">

</div> 

<div class="alert alert-info">
    
5. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, np.nan, np.nan, 95], 
        'Second Score': [30, np.nan, 45, 56], 
        'Third Score':[52, np.nan, 80, 98], 
        'Fourth Score':[60, 67, 68, 65]} 

    
<img align="left" width="150"  float= "none" align="middle" src="figuras/nan5.png">
    
</div> 